### **CrewAI is a cutting-edge framework for orchestrating autonomous AI agents**.

CrewAI enables you to create AI teams where each agent has specific roles, tools, and goals, working together to accomplish complex tasks.

Think of it as assembling your dream team - each member (agent) brings unique skills and expertise, collaborating seamlessly to achieve your objectives.


# Creating Agents to Reasearch and Write an Article

In [24]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

In [35]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [25]:
from crewai import Agent, Task, Crew
from langchain.chat_models import ChatOpenAI

In [26]:
import os
from google.colab import userdata

OPENAI_API_KEY = userdata.get("OPENAI_API_KEY")


# Creating Agents

## Planner Agent

In [27]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	  verbose=True,
    llm=ChatOpenAI(openai_api_key=OPENAI_API_KEY, model_name='gpt-4o')
)

## Writer Agent

In [28]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True,
    llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model_name='gpt-4o')
)


## Editor Agent

In [29]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True,
    llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model_name='gpt-4o')
)


# Creating Tasks

## Planning Task

In [30]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

## Writing Task

In [31]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

## Editing Task

In [32]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

# Creating the Crew

1. Create the crew of Agents

2. Pass the tasks to be performed by those agents.

**Note: For this simple example, the tasks will be performed sequentially (i.e they are dependent on each other), so the order of the task in the list matters.**

3. verbose=2 allows you to see all the logs of the execution.

In [ ]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

In [38]:
result = crew.kickoff(inputs={"topic": "Computer Vision"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Computer Vision.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
Thought: I have gathered enough information to create a comprehensive content plan on the topic of Computer Vision, including trends, audience analysis, an outline, SEO keywords, and resources.

Final Answer:

**Content Plan Document: Computer Vision**

**1. Introduction:**
   - Brief explanation of what computer vision is.
   - Importance and applications of computer vision in various industries.
   - Mention the continuous growth and evolution in the field.

**2. Prioritize the Latest Trends, Key Players, and Noteworthy News:**
   - **Latest Trends:**
     - D

In [40]:
from IPython.display import Markdown
Markdown(result)

```markdown
# Unveiling the Future: A Deep Dive into Computer Vision

## Introduction: The Rise of Computer Vision

Computer vision, a subfield of artificial intelligence, involves teaching machines to interpret and understand the visual world. By using digital images from cameras and videos and deep learning models, machines can accurately identify and classify objects — and then react to what they "see." The significance of computer vision extends across various industries, revolutionizing how businesses operate, enhancing productivity, and opening up new avenues for innovation. From healthcare to automotive, the continuous growth and evolution in the field of computer vision are crafting a future where machines not only see but perceive the world like humans.

Computer vision is transforming industries by automating processes and providing insights that were previously unattainable. In healthcare, it aids in early diagnosis through imaging, while in retail, it enhances customer experience through smart checkouts and personalized ads. This technology is becoming an integral part of modern systems, making it essential for businesses to adapt and integrate these advancements into their operations.

## Exploring Computer Vision Trends and Innovations

The latest trends in computer vision are redefining its capabilities and applications. Deep learning and neural networks have significantly advanced computer vision, enabling more accurate and efficient visual recognition systems. Real-time video processing and analysis are becoming increasingly crucial, especially in areas like surveillance and live sports analytics. Edge computing is another trend reshaping computer vision, allowing data processing at the edge of the network to reduce latency and enhance performance.

In autonomous vehicles and robotics, computer vision is pivotal, providing the "eyes" that guide machines in navigating complex environments. Additionally, the integration of AR/VR with computer vision technologies is creating immersive experiences that blend the digital and physical worlds seamlessly. These innovations are setting the stage for a future where the possibilities for computer vision applications are virtually limitless.

## Key Players and Technologies Shaping Computer Vision

Numerous companies and open-source projects are at the forefront of computer vision advancements. Tech giants like Google, Microsoft, Amazon, NVIDIA, and Intel are investing heavily in this field, developing cutting-edge technologies and solutions. These investments are driving rapid advancements, making sophisticated computer vision tools more accessible to a wider audience.

Open-source projects such as OpenCV, TensorFlow, and PyTorch are democratizing access to computer vision tools, enabling developers to innovate and experiment without high costs. Recent technological advancements have significantly enhanced computer vision capabilities, with breakthroughs in AI and machine learning pushing the boundaries of what's possible. Additionally, partnerships and acquisitions in the tech industry underscore the strategic importance of computer vision as companies seek to consolidate their positions and accelerate innovation.

## Addressing Challenges and Ethical Concerns

As computer vision technologies become more prevalent, several challenges and ethical concerns need to be addressed. Integrating computer vision with existing systems can be complex, requiring significant technical expertise and resources. Moreover, there are concerns about data privacy, as computer vision systems often process sensitive personal data.

Ethical considerations also arise around surveillance and the potential for bias in machine learning algorithms. Addressing these challenges requires a collaborative approach, involving not only technological solutions but also regulatory frameworks and ethical guidelines to ensure that computer vision technologies are used responsibly and for the public good. Proactive measures and transparent practices will be key in fostering trust and acceptance of these technologies.

## Future Prospects and Opportunities in Computer Vision

The future of computer vision is brimming with potential and opportunities. Emerging trends suggest that computer vision will continue to evolve, with applications expanding into new domains and industries. Businesses and professionals looking to leverage computer vision should focus on emerging technologies and stay updated with industry trends.

By doing so, they can harness the transformative power of computer vision to drive innovation, enhance efficiency, and gain a competitive edge. The field is ripe for exploration, offering numerous opportunities for those willing to invest in research and development, and to push the boundaries of what's possible. The ongoing advancements promise to unlock new capabilities, enabling a smarter and more connected world.

## Conclusion and Call to Action

In conclusion, computer vision is a transformative technology with the potential to revolutionize various industries. By understanding the latest trends, key players, and challenges, stakeholders can better navigate the complex landscape of computer vision. As the field continues to evolve, it is crucial for businesses and professionals to stay informed and proactive in exploring computer vision solutions.

To delve deeper into the world of computer vision, readers are encouraged to subscribe to our newsletter or join our upcoming webinar, where experts will share insights and discuss the future of this exciting field. Engaging with these resources will provide valuable knowledge and insights, empowering stakeholders to make informed decisions and capitalize on the opportunities that computer vision presents.
```

In [41]:
topic = "YOUR TOPIC HERE"
result = crew.kickoff(inputs={"topic": topic})

In [ ]:
Markdown(result)